## Energy Flow Tutorial

[EnergyFlow website](https://pkomiske.github.io/EnergyFlow/)

### Energy Flow Polynomials

In this tutorial, we introduce the EnergyFlow package for computing arbitrary Energy Flow Polynomials (EFPs) and collections of EFPs. The package is built with both usability, flexibility, and computational efficiency in mind.

For a collection of $M$ particles with energy measure $z_i$ and pairwise angular distance measures $\theta_{ij}$, the EFPs are multiparticle energy correlator observables, indexed by multigraphs $G$, and defined as:
$$ \mathrm{EFP}_G = \sum_{i_1=1}^M \sum_{i_2=1}^M\cdots \sum_{i_N=1}^M z_{i_1}z_{i_2}\cdots z_{i_N} \prod_{(k,\ell)\in G} \theta_{i_ki_\ell},$$
where $(k,\ell)\in G$ iterates over the edges in the multigraph.

### Choices of Measure

The specific choice of energy and angular measures depends on the collider context. We provide the following `measure` options (default is `hadrdot`):

`hadr`: $$ z_i = p_{T,i}^\kappa,\,\,\,\,\,\, \theta_{ij} = ((y_i - y_j)^2 + (\phi_i - \phi_j)^2)^{\beta/2} $$
`hadrdot`: $$ z_i = p_{T,i}^\kappa,\,\,\,\,\,\, \theta_{ij} = (2\, \hat p_i^\mu \hat p_{i\,\mu})^{\beta/2},\,\,\,\,\,\, \mathrm{where }\,\,\,\,\,\,\hat p_i^\mu \equiv p_i^\mu/p_{T,i}  $$
`ee`: $$ z_i = E_i^\kappa,\,\,\,\,\,\, \theta_{ij} = (2\, \hat p_i^\mu \hat p_{i\,\mu})^{\beta/2},\,\,\,\,\,\, \mathrm{where }\,\,\,\,\,\,\hat p_i^\mu \equiv p_i^\mu/E_i $$

The energy and angular weighting parameters `kappa` and `beta` default to $\kappa=1$ and $\beta = 2$. The choice of $\kappa = 1$ is required for infrared and collinear (IRC) safety of the observables. Any choice of $\beta > 0$ guarantees IRC safety. We also provide the `normed` option (default is `True`) to use a normalized and dimensionless energy measure $z_i/\sum_{j=1}^M z_j$ rather than $z_i$.

With this refresher, we have enough to begin using EnergyFlow to compute arbitrary EFPs! Ensure you have EnergyFlow installed. It is easily `pip` installable by executing:
` pip install energyflow `

We start by importing the EnergyFlow package as well as some other helpful Python libraries.

In [37]:
import energyflow as ef
import numpy as np

### Obtaining Events
Let's get some events. Typically these would be read in from your favorite event generator for your physics case of interest.

For the purposes of this tutorial, we will uniformly sample massless $M$-body phase space with the use of our implementation of the [RAMBO](https://www.sciencedirect.com/science/article/pii/0010465586901190?via%3Dihub) algorithm via the function `ef.gen_massless_phase_space`. It returns `nevents` events consisting of `nparticles` massless four-momenta with center of mass energy of `energy` in the center of momentum frame. In general, EnergyFlow supports events as arrays of four-momenta `[E,px,py,pz]` or arrays of hadronic coordinates `[pT,y,phi]` for hadronic measures.

Let's generate 50 events with 20 particles each at center of mass energy 100 GeV.

In [38]:
events = ef.gen_massless_phase_space(nevents=50, nparticles=20, energy=100)

### Defining Graphs

To specify a particular EFP to be computed for our events, we must simply specify the corresponding multigraph.


In EnergyFlow, multigraphs are specified as lists of edges, where edges are pairs of vertices. Here are several examples of graphs given as edge lists, where we label the vertices with integers from $0$ to $N-1$.

In [39]:
line     = [(0,1)]
wedge    = [(0,1), (0,2)]
triangle = [(0,1), (0,2), (1,2)]
square   = [(0,1), (1,2), (2,3), (3,1)]

Multigraphs can have multiple edges per pair of vertices. Here are several examples of multigraphs as edgelists, with one or more doubled edges.

In [40]:
multiline      = [(0,1), (0,1)]
multiwedge1    = [(0,1), (0,1), (0,2)]
multiwedge2    = [(0,1), (0,1), (0,2), (0,2)]
multitriangle1 = [(0,1), (0,2), (0,2), (1,2)]

In fact, arbitrary objects can be used to label the vertices. We typically use integer-labeling for simplicity and readibility, though this is not required. The following two edge lists both define the _same_ graph from the perspective of EnergyFlow.

In [41]:
graph_LHC  = [('atlas','cms'),('atlas','cms'),('atlas','lhcb'),('cms','lhcb'),('lhcb','alice')]
graph_ints = [(0,1),(0,1),(0,2),(1,2),(2,3)]

### Computing an Energy Flow Polynomial

EFPs are defined by passing a graph and measure choies to `ef.EFP`. The `compute` method of `ef.EFP` can then be called on an event in order to compute the EFP on that event.

For concreteness, let's begin by defining the EFP corresponding to the line graph `[(0,1)]` (one edge connecting two vertices) which should be equal to twice the squared center of mass energy of the event with a suitable choice of measure parameters.

In [42]:
# Computing a single EFP on a single event

# specify a graph and event
graph = [(0,1)]
event = events[0]

# define the EFP corresponding to the specified graph
EFP_graph = ef.EFP(graph, measure='hadr', beta=1, normed=False)

# compute the EFP on the specified event
result = EFP_graph.compute(event)

print("EFP Value:", result)

EFP Value: 10957.14461333734


We can see that the value of this EFP is indeed $\mathrm{EFP}_{[(0,1)]} = 2\times E_\mathrm{CM}^2 = 2\times (100\,\mathrm{GeV})^2 = 20\,000\, \mathrm{GeV}^2$, as expected for our events.

The framework above can be immediately extended to computing the value of an EFP on a collection of many events. This can either be done with list comprehension or even more simply using the `batch_compute` method, which tries to use as many processes as there are CPUs in the machine. The number of worker processes can be controlled by passing in `n_jobs`.

In [43]:
# Computing a single EFP on many events

# specify a graph
graph = [(0,1), (0,2), (0,2), (1,2)]

# define the EFP corresponding to the specified graph
efp_graph = ef.EFP(graph, measure='hadr', beta=1, normed=True)

# compute the EFP on the collection of events with list comprehension
results = np.asarray([efp_graph.compute(event) for event in events])
print("EFP w. list comprehension:", results)

EFP w. list comprehension: [13.1007147  13.32582473 15.87006697 12.17411306 12.26897294 21.80249016
 12.28479733 14.57455335 18.40761568  8.9176471  12.9372133  13.3031987
 12.6051932  16.10531546  9.6111098  12.64911466 12.18696638 12.78617212
 10.29566507 12.25617297  9.67355361 11.54286862 10.71722065 13.93871051
 17.55506044 19.2464684  13.87631094 14.06889131 17.73853677 15.72436017
 13.74849873 21.32696447 13.88874177 12.32863559 11.9475095  11.19274329
 10.42418362 16.7504581  17.23307003 14.72009819  8.8558485  19.26503688
 17.51708434 24.98725091  9.26292459 14.85920416 14.49473615 13.94631909
 12.61959745  9.692009  ]


### Computing a set of Energy Flow Polynomials

If we are interested in using the spanning properties of EFPs, we typically want to compute large numbers of EFPs. The EnergyFlow package does the heavy lifting for you! It contains information about all multigraphs with up to 10 edges, which can be easily and intuitively accessed using `EFPSet`.

Relevant graph quantities which can easily be used to select a set of multigraphs are summarized below. More options are available: for a full list see the [documentation](https://pkomiske.github.io/EnergyFlow/).

`n` : Number of vertices in the multigraph.

`d` : Degree, or number of edges in the multigraph.

`v` : Maximum valency (number of edges touching a vertex).

`c` : Variable Elimination computational complexity $\mathcal O(M^c)$

`p` : Number of prime factors (or connected components of the multigraph).

As a basis, EFPs are typically organized by the number of edges `d`. Not only does this correspond to the degree of the polynomial, but there are also a _finite_ number of multigraphs up to a specified degree `d`. Lets get  all EFPs with up to five edges by passing `d<=5` to `EFPSet` and compute them on our events!

In [44]:
# get all EFPs with d<=5 (up to d<=10 available by default)
efpset = ef.EFPSet('d<=5', measure='hadr', beta=1, normed=True, verbose=True)

# compute their values on our events
results = np.asarray([efpset.compute(event) for event in events])

print("Results:", results)

Originally Available EFPs:
  Prime: 23690
  Composite: 21540
  Total:  45230
Current Stored EFPs:
  Prime: 53
  Composite: 48
  Total:  101
Results: [[ 1.83179489  4.46479325 11.89117549 ... 27.44301875 20.93692254
  20.62453738]
 [ 1.85464566  4.27289615 10.73589544 ... 27.25870244 22.03681672
  21.94344121]
 [ 1.91751039  4.65693327 12.46788262 ... 32.83319851 26.25108899
  25.92320115]
 ...
 [ 1.86793375  4.38310892 11.24476367 ... 28.56713375 22.90718089
  22.74088392]
 [ 1.81223096  4.27527233 11.14108326 ... 25.44511519 19.7987956
  19.5464426 ]
 [ 1.72569245  3.85189594  9.49829239 ... 19.79542114 15.35691378
  15.30442436]]


We can very easily do much more sophisticated EFP selections using any of the graph quantities. For example to select all prime EFPs with at most 4 vertices and at most 5 edges, we simply use `EFPSet` with the following intuitive syntax:

In [45]:
# get all EFPs with n<=4, d<=5, that are prime (i.e. p==1)
efpset = ef.EFPSet(('n<=',6), ('d<=',5), ('p==',1), measure='hadr', beta=1, normed=True, verbose=True)

# compute their values on our events
results = np.asarray([efpset.compute(event) for event in events])

Originally Available EFPs:
  Prime: 23690
  Composite: 21540
  Total:  45230
Current Stored EFPs:
  Prime: 53
  Composite: 0
  Total:  53


To study the properties of an individual EFP within `EFPSet` we can easily do this using `specs` and `graphs`. Suppose we are interested in the 95th EFP. We can find out its graph and all of its relevant information simply with the following syntax. Here we show only a subset of all the information that specs provides about an EFP: for a full list see the [documentation](https://pkomiske.github.io/EnergyFlow/).

In [46]:
efpset = ef.EFPSet('d<=5', measure='hadr', beta=1, normed=True, verbose=True)

ind = 95
graph = efpset.graphs(ind)
n, _, d, v, _, c, p, _ = efpset.specs[ind]

print("Graph:", graph)
print("Number of vertices, n:", n)
print("Number of edges,    d:", d)
print("Maximum valency,    v:", v)
print("VE complexity,      c:", c)
print("Number of primes,   p:", p)

Originally Available EFPs:
  Prime: 23690
  Composite: 21540
  Total:  45230
Current Stored EFPs:
  Prime: 53
  Composite: 48
  Total:  101
Graph: [(0, 1), (2, 3), (4, 5), (4, 6), (4, 7)]
Number of vertices, n: 8
Number of edges,    d: 5
Maximum valency,    v: 3
VE complexity,      c: 2
Number of primes,   p: 3


### Custom measures

If you want to specify your own custom measure for $z_i$ and $\theta_{ij}$ to be used in the EFP formula, that's also possible within the EnergyFlow package. You simply compute your own custom-defined `zs` and `thetas` on the events and pass them to the `compute` methods.

We demonstrate this below by using random numbers for `zs` and `thetas`, which can be replaced with any custom values.

In [11]:
# zs and thetas can be passed in explicitly if you want to use a custom measure
(zs, thetas) = (np.random.rand(100,25), np.random.rand(100,25,25))

results = np.asarray([efpset.compute(zs=z, thetas=theta) for (z,theta) in zip(zs,thetas)])
print(results)

[[1.15186438e+02 7.69068035e+01 5.78015000e+01 ... 1.17535431e+08
  1.36898595e+09 2.02771417e+10]
 [9.64046894e+01 6.37018005e+01 4.73836346e+01 ... 5.70750350e+07
  6.08828884e+08 8.32704283e+09]
 [7.19416458e+01 4.84149099e+01 3.68815022e+01 ... 1.80268662e+07
  1.60204481e+08 1.92708932e+09]
 ...
 [6.59271470e+01 4.36015963e+01 3.23252405e+01 ... 1.24938198e+07
  1.09820619e+08 1.24543599e+09]
 [7.77016944e+01 5.08128653e+01 3.73904956e+01 ... 2.38377441e+07
  2.31418239e+08 2.83238605e+09]
 [5.76998276e+01 3.85124726e+01 2.91119256e+01 ... 7.39818094e+06
  6.08408480e+07 6.39547164e+08]]


And that's it! Now you should be able to specify any EFP (i.e. multigraph) or set of EFPs that you want to comput with `EFP` or `EFPSet`, compute them on a set of events with `compute` or `batch_compute`, and study the results with `specs` and `graphs`! As always, see the [documentation](https://pkomiske.github.io/EnergyFlow/) for a full description of the EnergyFlow package.